In [2]:
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
import pandas as pd
import csv
import os 

In [3]:
df1 = pd.read_csv("vtdatabrokers2019-Jan-18-2023.csv") # reading in privacy policy corpus
df1 = df1.head()

In [4]:
# read in the dataframe and then go row by row, reading for a specific column
corpusurls = [] # sets first parameter for the dataframe
column_of_links = [] # sets second parameter for the dataframe
counts = [] # sets third parameter for the dataframe
for index, row in df1.iterrows(): # read for each row in the csv file
    try:
        response=requests.get(row['PrivacyPolicyInternetAddress']) # identify website's scrapeability (for first URL under current policy column)
        response.status_code=200 # set condition for the status code to be equal to 200; signifies website is accessible and scrapeable
        corpusurls.append(row['PrivacyPolicyInternetAddress']) # if website is useable, append url to corpus urls list
        page = urlopen(row['PrivacyPolicyInternetAddress']) # open the page to begin reading into the links
        html_bytes = page.read() # reads html file of page variable
        html = html_bytes.decode("utf-8") # decodes html from utf-8 into readable text for python?
        soup = BeautifulSoup(html, "html.parser") # create a beautifulsoup object where html is parsed and output is stored as a variable
        links = soup.find_all("a") # create list object that contains all the "a" tagged information within the parsed html file
        for link in links: # for loop to evaluate which links may count as nested links
            usable_urls = [] # create a list object to store usable links
            url = link.get('href') # define the characteristic of the link to be "href" tagged 
            if url is None or "#" in url: # if url is not found or there is a # in the url, then continue onto the next link
                continue
            elif "mailto:" in url: # if url is an email link, then continue onto the next link
                continue
            else:
                usable_urls.append(url) # append the URL to the usable links list
        link_count = len(usable_urls) # count the number of links in the list and store that as a count for the URL
        counts.append(link_count) # store link count for the URL as an item in the count list
        column_of_links.append(usable_urls) # store the list of links as an item within the column of link lists
    except: # if status code is not 200, skip to the next corpus URL
         continue
df2=pd.DataFrame(corpusurls, column_of_links, counts, columns=[['URLS','Lists', 'Counts']]) # create the dataframe to display counts of links for each policy read in
df2.to_csv('Link_Counts_V1.csv', index=True) # save the dataframe to a csv file
print("Success")



TypeError: DataFrame.__init__() got multiple values for argument 'columns'